In [9]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# LCLS Classic model 

In [18]:
from lcls_live.bmad import LCLSTaoModel
from lcls_live.epics import epics_proxy

import numpy as np
import os
import json
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
# Make sure this exists
assert 'LCLS_CLASSIC_LATTICE' in os.environ

In [20]:
# Optional.
# For archiver, if off-site

# Open an SSH tunnel in a terminal like:
# ssh -D 8080 cmayes@rhel6-64.slac.stanford.edu 
# And then set:
os.environ['http_proxy']='socks5h://localhost:8080'
os.environ['HTTPS_PROXY']='socks5h://localhost:8080'
os.environ['ALL_PROXY']='socks5h://localhost:8080'

# Get snapshot

In [21]:
# Cached EPICS pv data
epics = epics_proxy('data/epics_snapshot_2018-03-06T11:22:45.000000-08:00.json', verbose=True)

Loaded data/epics_snapshot_2018-03-06T11:22:45.000000-08:00.json with 1722 PVs


In [22]:
M = LCLSTaoModel('lcls_classic', epics = epics,verbose=True, ploton=True)
print(M)

Initialized Tao with /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmps2loiv4r/tao/tao.init
Loading all settings
Reading: settings/klystron_settings.bmad
Written: /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmps2loiv4r/tao/settings/klystron_settings.bmad
Reading: settings/linac_settings.bmad
Written: /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmps2loiv4r/tao/settings/linac_settings.bmad
Reading: settings/collimator_settings.bmad
Written: /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmps2loiv4r/tao/settings/collimator_settings.bmad
Calling: settings/LEM_settings.tao
Written: /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmps2loiv4r/tao/settings/LEM_settings.tao
offsetting bunch compressors
call /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmps2loiv4r/tao/scripts/BC_offsets.tao
LEMing
call /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmps2loiv4r/tao/scripts/LEM.tao
Configured.
_______________________________________________
___________________________________

In [23]:
%%tao
place floor beta_compare
set lattice base = model

-------------------------
Tao> place floor beta_compare
-------------------------
Tao> set lattice base = model
-------------------------
Tao> 


In [24]:
# Restore from some other time
M.archiver_restore('2018-11-06T11:22:45.000000-08:00')

Requesting: http://lcls-archapp.slac.stanford.edu/retrieval/data/getDataAtTime?at=2018-11-06T11:22:45.000000-08:00&includeProxies=true
Loading all settings
Reading: settings/klystron_settings.bmad
Written: /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmps2loiv4r/tao/settings/klystron_settings.bmad
Reading: settings/linac_settings.bmad
Written: /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmps2loiv4r/tao/settings/linac_settings.bmad
Reading: settings/collimator_settings.bmad
Written: /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmps2loiv4r/tao/settings/collimator_settings.bmad
Calling: settings/LEM_settings.tao
Written: /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmps2loiv4r/tao/settings/LEM_settings.tao
offsetting bunch compressors
call /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmps2loiv4r/tao/scripts/BC_offsets.tao
LEMing
call /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmps2loiv4r/tao/scripts/LEM.tao


## Track particles

In [ ]:
%%tao
set beam_init beam_track_end = UEEND
set csr_param n_bin = 40
snparticle 10000
beamon
beamoff

In [ ]:
h5file1 = os.path.join(os.getcwd(), 'test1.h5')
M.cmd('write beam -at UEBEG '+h5file1)

h5file2 = os.path.join(os.getcwd(), 'test2.h5')
M.cmd('write beam -at UEEND '+h5file2)

# Plot

In [ ]:
from opmd_beamphysics import bunch_plotting, bunch_tools, bunch_stats
from bokeh.plotting import  show, output_notebook
from bokeh.layouts import column, row
output_notebook(verbose=False, hide_banner=True)
from h5py import File

In [ ]:
#h1.close()
#h2.close()
h1 = File(h5file1, 'r')
h2 = File(h5file2, 'r')
bunch1 = h1['data/00001/particles']
bunch2 = h2['data/00001/particles']

show(
    row(
        column(    
            bunch_plotting.plot_bunch_h5(bunch1, 't', 'pz', liveOnly=True,  bins = 100),
            bunch_plotting.plot_histogram_h5(bunch1, 't', bins=100)), 
    column(
        
    bunch_plotting.plot_bunch_h5(bunch2, 't', 'pz', liveOnly=True,  bins = 100),
    bunch_plotting.plot_histogram_h5(bunch2, 't', bins=100))        

    )
    
    )

In [ ]:
# Cleanup
os.remove(h5file1), os.remove(h5file2)